In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## Step 0.0: Create the output directory

import os

def create_gpkgs():
    """
    Creates <base>/RiverJoin/output_gpkg/ next to this script (or in cwd if __file__ is undefined),
    and ensures placeholder GeoPackage files exist there:
      - TempPoints.gpkg
      - TempReaches.gpkg
      - TempReachesBuffers.gpkg
      - TempReachesPointsPerp.gpkg
      - FinalReaches.gpkg
      - FinalReachesWithJoinedCommonID.gpkg

    Returns a dict mapping each filename to its full path.
    """
    # 1) # Determine base directory: script folder if running as a file, else cwd
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()

    # 2) Build our output base directory under "output"
    base_dir = os.path.join(script_dir, "RiverJoin", "output")
    os.makedirs(base_dir, exist_ok=True)

    # 3) Define the GeoPackage filenames
    gpkg_names = [
        "TempPoints.gpkg",
        "TempReaches.gpkg",
        "TempReachesBuffers.gpkg",
        "TempReachesPointsPerp.gpkg",
        "FinalReaches.gpkg",
        "FinalReaches_JoinedCommonID.gpkg",
        # "FinalReaches_AggregatedCommonID.gpkg"
    ]

    # 4) Ensure each file’s parent directory exists (all under base_dir)
    paths = {}
    for name in gpkg_names:
        full_path = os.path.join(base_dir, name)
        # base_dir already exists, but safe to call again
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        print(f"Ensured existence of directory: {full_path}")
        paths[name] = full_path

    return paths

# Example usage:
if __name__ == "__main__":
    gpkg_paths = create_gpkgs()
    output_point_gpkg              = gpkg_paths["TempPoints.gpkg"]
    output_reach_gpkg              = gpkg_paths["TempReaches.gpkg"]
    output_reach_buffer_gpkg       = gpkg_paths["TempReachesBuffers.gpkg"]
    output_reach_point_perp_gpkg   = gpkg_paths["TempReachesPointsPerp.gpkg"]
    output_final_reach_gpkg        = gpkg_paths["FinalReaches.gpkg"]
    output_final_reach_joined_gpkg = gpkg_paths["FinalReaches_JoinedCommonID.gpkg"]
    # output_final_reach_aggregated_gpkg = gpkg_paths["FinalReaches_AggregatedCommonID.gpkg"]

In [ ]:
import geopandas as gpd
from typing import Optional

## Step 0.1: Clip target flowlines by a VPU boundary or, if none provided, by the envelope of join flowlines
def clip_flowlines_by_boundary(
    join_fl_gpkg: str,
    join_fl_layer: str,
    target_fl_gpkg: str,
    target_fl_layer: str,
    output_clip_gpkg: str,
    output_clip_layer: str,
    vpu_boundary_gpkg: Optional[str] = None,
    vpu_boundary_layer: Optional[str] = None
) -> str:
    """
    Clips the target flowline layer by a boundary polygon. If a VPU boundary is supplied,
    uses that polygon; otherwise computes the bounding envelope of the join flowlines
    and uses it as the clipping polygon. Writes the result into a GeoPackage.

    Parameters:
    -----------
    join_fl_gpkg : str
        GeoPackage path containing the join flowlines (used to compute envelope if needed).
    join_fl_layer : str
        Layer name inside join_fl_gpkg.
    target_fl_gpkg : str
        GeoPackage path containing the target flowlines to clip.
    target_fl_layer : str
        Layer name inside target_fl_gpkg.
    output_clip_gpkg : str
        GeoPackage path where the clipped result will be written.
    output_clip_layer : str
        Layer name to use inside output_clip_gpkg for the clipped result.
    vpu_boundary_gpkg : Optional[str]
        GeoPackage path containing an explicit VPU boundary polygon.
    vpu_boundary_layer : Optional[str]
        Layer name inside vpu_boundary_gpkg.

    Returns:
    --------
    str
        The path to output_clip_gpkg containing the new clipped layer.
    """
    # 1. Read join flowlines (for envelope fallback)
    join_gdf = gpd.read_file(join_fl_gpkg, layer=join_fl_layer)

    # 2. Determine clipping boundary
    if vpu_boundary_gpkg and vpu_boundary_layer:
        boundary_gdf = gpd.read_file(vpu_boundary_gpkg, layer=vpu_boundary_layer)
    else:
        # Compute envelope of all join flowlines
        combined = join_gdf.geometry.union_all()
        envelope = combined.envelope
        boundary_gdf = gpd.GeoDataFrame({'geometry': [envelope]}, crs=join_gdf.crs)

    # 3. Read target flowlines
    target_gdf = gpd.read_file(target_fl_gpkg, layer=target_fl_layer)

    # 4. Reproject target to match boundary CRS if necessary
    if target_gdf.crs != boundary_gdf.crs:
        target_gdf = target_gdf.to_crs(boundary_gdf.crs)

    # 5. Perform clipping
    clipped = gpd.clip(target_gdf, boundary_gdf)

    # 6. Write clipped result
    clipped.to_file(output_clip_gpkg, layer=output_clip_layer, driver="GPKG", mode='w')

    # 7. Notify user of output
    print(f"Step 0.1: Clipped flowlines written to: {output_clip_gpkg}/{output_clip_layer}")

    return clipped

In [ ]:
## Step 1: Extracts join flowlines whose midpoints lie within a buffer around target flowlines. 

import geopandas as gpd
import os
from shapely.geometry import LineString

def extract_stream_network_within_sword_buffer(
    huc_id: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    target_fl_gpkg: str,
    target_fl_layer: str,
    unique_id: str,
    output_point_gpkg: str,
    output_reach_buffer_gpkg: str,
    output_reach_gpkg: str,
    buffer_distance: int = 100,  # meters, default buffer distance
) -> str:
    """
    Reads join & target from GeoPackages,
    buffers the target, selects midpoints within that buffer, and writes
    both midpoints and extracted reaches into GeoPackages, appending layers.
    """
    # Read join flowlines
    join_fl = gpd.read_file(join_fl_gpkg, layer=join_fl_layer)
    # Read target flowlines
    target_fl = gpd.read_file(target_fl_gpkg, layer=target_fl_layer)

    # Reprojection logic:
    # 1) If target_fl is already projected → no operation.
    # 2) Else if join_fl is projected → project target_fl to join_fl CRS.
    # 3) Else → project target_fl to EPSG:3857.
    if target_fl.crs.is_projected:
        print("Step 1: Situation 1: target_fl CRS is projected; no reprojection needed.")
    # elif join_fl.crs.is_projected:
    #     print("Step 1: Situation 2: join_fl CRS is projected; reprojecting target_fl to join_fl CRS.")
    #     target_fl = target_fl.to_crs(join_fl.crs)
    else:
        # compute UTM zone from target_fl extent
        minx, miny, maxx, maxy = target_fl.total_bounds
        midx, midy = (minx + maxx) / 2, (miny + maxy) / 2
        zone = int((midx + 180) // 6) + 1
        epsg_utm = 32600 + zone if midy >= 0 else 32700 + zone
        target_fl = target_fl.to_crs(epsg=epsg_utm)
        print(f"Step 1: Situation 3: neither CRS is projected; reprojecting target_fl to UTM zone EPSG:{epsg_utm}")

    # Compute midpoints
    def midpoint_of_line(geom):
        if geom.geom_type == "MultiLineString":
            line = list(geom.geoms)[0]
        else:
            line = geom
        return line.interpolate(0.5, normalized=True)

    midpoints = join_fl.geometry.apply(midpoint_of_line)
    midpoints_gdf = gpd.GeoDataFrame(
        {unique_id: join_fl[unique_id], "orig_index": join_fl.index},
        geometry=midpoints,
        crs=join_fl.crs
    )

    # Write midpoints to GeoPackage
    mid_layer = f"join_fl_midpoints_{huc_id}"
    midpoints_gdf.to_file(
        output_point_gpkg,
        layer=mid_layer,
        driver="GPKG",
        mode="w"
    )

    # Buffer the target flowlines
    buffered = target_fl.copy()
    buffered.geometry = buffered.geometry.buffer(buffer_distance)

    # Write buffer to GeoPackage
    buf_layer = f"target_fl_buffer_{int(buffer_distance)}m"
    buffered.to_file(
        output_reach_buffer_gpkg,
        layer=buf_layer,
        driver="GPKG",
        mode="w"
    )

    # Select midpoints within buffer
    selected = gpd.sjoin(midpoints_gdf, buffered, how="inner", predicate="within")

    # Extract join flowlines by midpoint indices
    extracted_indices = selected["orig_index"].unique()
    extracted = join_fl.loc[extracted_indices].copy()

    # Write extracted reaches to GeoPackage
    extracted_layer = f"join_fl_extracted_reaches_{huc_id}"
    extracted.to_file(
        output_reach_gpkg,
        layer=extracted_layer,
        driver="GPKG",
        mode="w"
    )

    print(f"    Extracted flowlines saved to {output_reach_gpkg}/{extracted_layer}")
    return extracted

In [ ]:
## Step 2: Reconstruct disconnected segments

import geopandas as gpd
import pandas as pd
import numpy as np
import os
import fiona

def reconstruct_disconnected_segments(
    unique_id: str,
    unique_id_nextDown: str,
    extracted_layer: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    huc_id: str,
    output_reach_gpkg: str,
) -> str:
    """

    1) Reads the initially extracted network (extract_gdf) and the full join flowline network (join_fl).
    2) Finds NextDownIDs (unique_id_nextDown) that do not appear as HydroIDs in the extracted set.
    3) Traces upstream from each “disconnected end,” adding any missing reaches until hitting a node that is already in the extracted set or a dead-end.
    4) Exports those “missing_reaches” to output_reach_gpkg/missing_reaches_{huc_id}.
    5) Merges them back into the extracted network, writing the final reconstructed network to output_reach_gpkg/output_final_reach_gpkg_{huc_id}.
    
    Returns:
        Path to the final reconstructed feature class GeoPackage.
    """
    # Step 0: Read inputs
    extract_gdf = gpd.read_file(output_reach_gpkg, layer=extracted_layer)
    orig_gdf    = gpd.read_file(join_fl_gpkg, layer=join_fl_layer)

    hydro_ids     = extract_gdf[unique_id].to_numpy()
    nextdown_ids  = extract_gdf[unique_id_nextDown].to_numpy()

    # Filter out null NextDownIDs
    valid_mask     = pd.notnull(nextdown_ids)
    nextdown_valid = nextdown_ids[valid_mask]

    disconnected_ends = np.unique(nextdown_valid[~np.isin(nextdown_valid, hydro_ids)])
    disconnected_segments = []

    # Step 1: Trace downstream from each disconnected end
    for end_id in disconnected_ends:
        current_id     = end_id
        missing_reaches = []

        while pd.notnull(current_id):
            match = orig_gdf[orig_gdf[unique_id] == current_id]
            if match.empty:
                break

            if current_id not in hydro_ids:
                missing_reaches.append(current_id)

            # Advance downstream
            current_id = match.iloc[0][unique_id_nextDown]
            if pd.isnull(current_id) or (current_id in hydro_ids):
                break

        if missing_reaches:
            disconnected_segments.append(missing_reaches)

    # Step 2: Export missing reaches
    all_missing_ids = [fid for seg in disconnected_segments for fid in seg]
    if all_missing_ids:
        missing_mask        = orig_gdf[unique_id].isin(all_missing_ids)
        missing_reaches_gdf = orig_gdf[missing_mask].copy()
    else:
        missing_reaches_gdf = orig_gdf.iloc[0:0].copy()  # empty

    missing_reaches_fc = f"join_fl_traced_reaches_{huc_id}"
    missing_reaches_gdf.to_file(output_reach_gpkg, 
                                layer=missing_reaches_fc,
                                driver="GPKG",
                                mode="w")

    # Step 3: Merge and write final
    merged_gdf = gpd.GeoDataFrame(
        pd.concat([extract_gdf, missing_reaches_gdf], ignore_index=True),
        crs=extract_gdf.crs
    )

    join_fl_traced_extracted = f"join_fl_traced_extracted_reaches_{huc_id}"

    # 1) If that layer already exists in the GeoPackage, delete it
    existing = fiona.listlayers(output_reach_gpkg)
    if join_fl_traced_extracted in existing:
        fiona.remove(output_reach_gpkg, layer=join_fl_traced_extracted)
        # print(f"Removed existing layer '{join_fl_traced_extracted}' from {output_reach_gpkg}")
    
    merged_gdf.to_file(output_reach_gpkg,
                    layer=join_fl_traced_extracted,
                    driver="GPKG",
                    mode="w")

    print(f"Step 2: Saved reconstructed reaches to {output_reach_gpkg}/{join_fl_traced_extracted}")
    return merged_gdf

In [ ]:
## Step 3: check the unnormal situations

import geopandas as gpd
import os

def process_flowlines(
    extract_stream_gpkg: str,
    extract_layer: str,
    traced_stream_gpkg: str,
    traced_layer: str,
    original_gpkg: str,
    original_layer: str,
    output_final_reach_gpkg: str,
    huc_id: str
) -> str:
    """
    Step 3: Identify extraction situation and handle “all_extracted” case.
    
    1) Reads the extracted (via buffer) and traced networks, plus the full original network from GeoPackages.
    2) Counts features in each.
    3) If extract count == 0 → no intersection.
       If traced count == extract count → all extracted; (# no dedupe) and write to output_final_reach_gpkg.
       Else → need upstream extraction.
    Returns the situation string.
    """
    # Read layers from GeoPackages
    extract_gdf = (
        gpd.read_file(extract_stream_gpkg, layer=extract_layer)
        .drop_duplicates(subset="geometry").reset_index(drop=True)
    )
    traced_gdf  = (
        gpd.read_file(traced_stream_gpkg, layer=traced_layer)
        .drop_duplicates(subset="geometry").reset_index(drop=True)
    )
    
    # orig_gdf    = (
    # gpd.read_file(original_gpkg, layer=original_layer)
    # .drop_duplicates(subset="geometry").reset_index(drop=True)
    # )

    count_extract  = len(extract_gdf)
    count_trace    = len(traced_gdf)
    # count_original = len(orig_gdf)

    if count_extract == 0:
        situation = "no_intersection"
        print("Step 3: There may be no intersection of extracted and target flowlines in this VPU.")
        return situation

    # elif count_trace == count_extract:
    #     situation = "all_extracted"

    #     # Write the raw extracted GeoDataFrame without deduplication
    #     output_layer = f"join_fl_final_reaches_{huc_id}"
    #     extract_gdf.to_file(
    #         output_final_reach_gpkg,
    #         layer=output_layer,
    #         driver="GPKG",
    #         mode="w"
    #     )

    #     print(f"Step 3: All flowlines extracted → {output_final_reach_gpkg} (layer: {output_layer})")
    #     return situation

    else:
        situation = "extract_upstream_others"
        print("Step 3: Upstream and other flowlines still need extraction (proceed to Step 4).")
        return situation

In [ ]:
### Step 4: Extract upstream and other flowlines
## Step 4.0.1: Create the spaced nodes along the flowlines

import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from pyproj import CRS

def float_to_tag(x: float) -> str:
    # use general format so small numbers don't go to scientific notation
    s = f"{x:g}"
    return s.replace(".", "p")

def extract_spaced_nodes_projected(
    target_reach_gpkg: str,
    target_reach_layer: str,
    target_reach_unique_id_field: str,
    output_reach_point_perp_gpkg: str,
    output_layer: str,
    spacing: float = 200  # meters,
) -> gpd.GeoDataFrame:
    """
    1) Read `target_reach_layer` from `target_reach_gpkg`.
    2) If geographic, reproject to a suitable UTM zone (meters);
       otherwise stay in the native projected CRS.
    3) Interpolate points every `spacing` meters along each segment,
       dropping the overlapping boundary point.
    4) Drop exact duplicates.
    5) If we reprojected in step 2, reproject back to original CRS,
       then write to `output_reach_point_perp_gpkg`/`output_layer`.
    """
    # 1) Load and clean
    lines = (
        gpd.read_file(target_reach_gpkg, layer=target_reach_layer)
           .dropna(subset=["geometry"])
           .explode(ignore_index=True)
    )
    orig_crs = lines.crs

    # print(f"Original CRS: {orig_crs}")

    # 2) Decide whether to reproject into UTM
    if orig_crs.is_geographic:
        # pick UTM from centroid
        minx, miny, maxx, maxy = lines.total_bounds
        midx, midy = (minx + maxx) / 2, (miny + maxy) / 2
        zone = int((midx + 180)//6) + 1
        epsg_utm = 32600 + zone if midy >= 0 else 32700 + zone
        lines_proj = lines.to_crs(epsg=epsg_utm)
    else:
        # already in a projected CRS (assumed meters)
        lines_proj = lines

    # print(f"Projected CRS: {lines_proj.crs}")

    # 3) Build spaced nodes in projected CRS
    point_records = []
    for row in lines_proj.itertuples():
        geom = row.geometry
        parts = geom.geoms if isinstance(geom, MultiLineString) else [geom]
        for part_idx, seg in enumerate(parts):
            L = seg.length
            if L == 0:
                continue
            # distances [0, spacing, 2*spacing, …, L]
            n_steps = max(1, int(np.ceil(L / spacing)))
            dists = np.linspace(0, L, n_steps + 1)
            if part_idx > 0:
                # drop the duplicate 0 m point on subsequent parts
                dists = dists[1:]
            for d in dists:
                pt = seg.interpolate(d)
                point_records.append({
                    "reach_id": getattr(row, target_reach_unique_id_field, None),
                    "geometry": pt
                })

    pts_proj = gpd.GeoDataFrame(point_records, crs=lines_proj.crs)

    # 4) Drop any exact duplicates
    pts_proj = pts_proj.drop_duplicates(subset="geometry").reset_index(drop=True)

    # print(f"Points original CRS: {pts_proj.crs}")

    os.makedirs(os.path.dirname(output_reach_point_perp_gpkg), exist_ok=True)
    pts_proj.to_file(
        filename=output_reach_point_perp_gpkg,
        layer=output_layer,
        driver="GPKG"
    )

    print(f"Step 4.0.1: Generated and wrote {len(pts_proj)} nodes (spacing={float_to_tag(spacing)}m) to {output_reach_point_perp_gpkg}/{output_layer}")
    return pts_proj

In [ ]:
## Step 4.0.2: Generate perpendicular lines with a specified length at each node

import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString

def generate_perpendicular_line(node: Point, previous_node: Point, length: float) -> LineString:
        
    """
    Generate a line perpendicular to the segment formed by `previous_node` and `node`.
    
    Parameters:
    - node: The current shapely Point
    - previous_node: The previous shapely Point
    - length: Total length of the perpendicular line (in coordinate units)
    
    Returns:
    - A shapely LineString, or None if the two points coincide.
    """
    x1, y1 = previous_node.x, previous_node.y
    x2, y2 = node.x, node.y
    dx, dy = x2 - x1, y2 - y1
    # perpendicular vector
    perp_dx, perp_dy = -dy, dx
    norm = np.hypot(perp_dx, perp_dy)
    if norm == 0:
        return None
    perp_dx /= norm
    perp_dy /= norm
    # endpoints
    start = (node.x + perp_dx * length/2, node.y + perp_dy * length/2)
    end   = (node.x - perp_dx * length/2, node.y - perp_dy * length/2)
    return LineString([start, end])

def extract_perpendicular_lines(
    nodes_gpkg: str,
    nodes_layer: str,
    output_reach_point_perp_gpkg: str,
    perp_line_length: float = 600,  # meters
    spacing: float = 200  # meters,
) -> gpd.GeoDataFrame:
    """
    1) Read `nodes_layer` from `nodes_gpkg`.
    2) Generate length-`perp_line_length` perpendiculars at each consecutive node pair.
    3) Clean invalid or degenerate lines.
    4) Write to `output_reach_point_perp_gpkg` (GeoPackage) under layer "<nodes_layer>_perp".
    """
    # 1) Read & clean nodes
    nodes = gpd.read_file(nodes_gpkg, layer=nodes_layer)
    nodes = nodes.dropna(subset=["geometry"])
    nodes = nodes[~nodes.geometry.is_empty]

    # 2) Build perpendiculars
    perp_lines = []
    for i in range(1, len(nodes)):
        cur = nodes.geometry.iloc[i]
        prev = nodes.geometry.iloc[i-1]
        line = generate_perpendicular_line(cur, prev, perp_line_length)
        if line is not None:
            perp_lines.append(line)

    perp_gdf = gpd.GeoDataFrame(geometry=perp_lines, crs=nodes.crs)

    # 3) Clean invalid or zero-length lines
    def valid_or_none(geom):
        if geom is not None and geom.is_valid and np.isfinite(geom.length) and geom.length > 0:
            return geom
        return None

    perp_gdf["geometry"] = perp_gdf["geometry"].apply(valid_or_none)
    perp_gdf = perp_gdf.dropna(subset=["geometry"]).reset_index(drop=True)

    # 4) Write out to GeoPackage
    perp_layer = f"perp_lines_len{float_to_tag(perp_line_length)}_space{float_to_tag(spacing)}m_{huc_id}"
    perp_gdf.to_file(
        filename=output_reach_point_perp_gpkg,
        layer=perp_layer,
        driver="GPKG"
    )
    print(f"Step 4.0.2: Saved {len(perp_gdf)} perpendicular lines to {output_reach_point_perp_gpkg}/{perp_layer}")

    return perp_gdf

In [ ]:
## Step 4.1: Remove perpendicular lines those intersect with already extracted flowlines

def remove_touching_perpendiculars(
    output_reach_gpkg: str,
    traced_layer: str,
    output_reach_point_perp_gpkg: str,
    perp_layer: str,
    huc_id: str
) -> str:
    """
    Pure-Python replacement for remove_touching_perpendiculars.

    1) Reads the traced flowlines from output_reach_gpkg/traced_layer
       and the perpendicular lines from output_reach_point_perp_gpkg/perp_layer.
    2) Performs an “intersects” spatial join to find which perp lines touch the traced reaches.
    3) Drops those intersecting lines.
    4) Writes out the remaining perp lines to 
       output_reach_point_perp_gpkg/perp_lines_notcross_{huc_id}.

    Returns:
        The name of the layer "perp_lines_notcross_{huc_id}".
    """

    # Read the traced flowlines
    traced_gdf = gpd.read_file(output_reach_gpkg, layer=traced_layer)
    # Read the perpendicular lines
    perp_gdf   = gpd.read_file(output_reach_point_perp_gpkg, layer=perp_layer)

    # Align CRS if needed
    if traced_gdf.crs != perp_gdf.crs:
        perp_gdf = perp_gdf.to_crs(traced_gdf.crs)

    # Find intersecting perp lines
    joined = gpd.sjoin(
        perp_gdf,
        traced_gdf,
        how="inner",
        predicate="intersects"
    )
    intersecting_idx = joined.index.unique()

    # Drop intersecting lines
    if len(intersecting_idx) > 0:
        perp_notcross_gdf = perp_gdf.drop(index=intersecting_idx)
    else:
        perp_notcross_gdf = perp_gdf.copy()

    # Define output layer name
    output_layer = f"perp_lines_notcross_{huc_id}"

    # Write remaining lines into the GeoPackage
    perp_notcross_gdf.to_file(
        output_reach_point_perp_gpkg,
        layer=output_layer,
        driver="GPKG"
    )

    print(f"Step 4.1: Wrote non‐crossing perpendicular lines to {output_reach_point_perp_gpkg}/{output_layer}")
    return output_layer

In [ ]:
## Step 4.2: Spatial join between all original target flowlines and remaining perpendicular lines

import geopandas as gpd
import pandas as pd
import os

def perform_spatial_join_with_perp_and_filter(
    huc_id: str,
    original_gpkg: str,
    original_layer: str,
    perp_gpkg: str,
    perp_notcross_layer: str,
    unique_id: str,
    output_reach_point_perp_gpkg: str,
    min_join_count: int = 5,
    strm_order_field: str = None,
    min_strm_order: int = 2
) -> gpd.GeoDataFrame:
    """
    1. One‐to‐many spatial join (intersects).
    2. Compute Join_Count for each original reach.
    3. Collapse to one‐to‐one (first match).
    4. Write joined (with Join_Count).
    5. Filter by:
         Join_Count >= min_join_count
         AND <strm_order_field> > min_strm_order
    6. Write filtered result.

    Parameters:
    - min_join_count:    minimum number of perp intersections to keep
    - strm_order_field:  name of the stream order attribute in your dataset
    - min_strm_order:    minimum value of strm_order_field to keep (strictly greater)
    """
    # 1) Read inputs
    orig_gdf = gpd.read_file(original_gpkg, layer=original_layer)
    perp_gdf = gpd.read_file(perp_gpkg,    layer=perp_notcross_layer)

    # 2) Align CRS
    if orig_gdf.crs != perp_gdf.crs:
        perp_gdf = perp_gdf.to_crs(orig_gdf.crs)

    # 3) One‐to‐many join
    joined_1toN = gpd.sjoin(
        orig_gdf,
        perp_gdf,
        how="left",
        predicate="intersects"
    )
    # restore original geometry if needed
    if "geometry_left" in joined_1toN.columns:
        joined_1toN = (
            joined_1toN
            .set_geometry("geometry_left")
            .drop(columns=["geometry_right"], errors="ignore")
        )

    # 4) keep only matched
    joined_filtered = joined_1toN[joined_1toN["index_right"].notna()].copy()

    # 5) compute Join_Count per original index
    counts = joined_filtered.groupby(joined_filtered.index).size()
    joined_filtered["Join_Count"] = joined_filtered.index.to_series().map(counts).astype(int)

    # 6) collapse to first match per unique_id
    joined_1to1 = joined_filtered.drop_duplicates(subset=unique_id, keep="first")

    # 7) write the joined-with-counts layer
    layer1 = f"orig_join_perp_notcross_intersected_{huc_id}"
    joined_1to1.to_file(
        output_reach_point_perp_gpkg,
        layer=layer1,
        driver="GPKG"
    )

    # 8) apply filter based on parameters
    if strm_order_field:
        filtered_gdf = joined_1to1[
            (joined_1to1["Join_Count"] >= min_join_count) &
            (joined_1to1[strm_order_field]  >  min_strm_order)
        ]
    else:
        filtered_gdf = joined_1to1[
            joined_1to1["Join_Count"] >= min_join_count
        ]

    # 9) write the filtered result
    layer2 = f"orig_join_perp_notcross_intersected_filtered_{huc_id}"
    filtered_gdf.to_file(
        output_reach_point_perp_gpkg,
        layer=layer2,
        driver="GPKG"
    )

    # Build a human‐readable filter description
    if strm_order_field:
        cond_txt = f"Join_Count≥{min_join_count} & {strm_order_field}>{min_strm_order}"
    else:
        cond_txt = f"Join_Count≥{min_join_count}"

    print(
        f"Step 4.2: Wrote {len(filtered_gdf)} further found flowlines "
        f"({cond_txt}) to {output_reach_point_perp_gpkg}/{layer2}"
    )

    return filtered_gdf

In [ ]:
## Step 4.3: Merge the extracted, traced, and intersected. And drop duplicates

def merge_and_drop_duplicates(
    join_fl_perp_intersected_filtered: str,
    join_fl_traced_extracted: str,
    original_gpkg: str,
    original_layer: str,
    output_reach_point_perp_gpkg: str,
    output_reach_gpkg: str,
    huc_id: str
) -> str:
    """
    1) Reads join_fl_perp_intersected_filtered and join_fl_traced_extracted into GeoDataFrames.
    2) Merges them row-wise.
    3) Drops duplicates by ["LINKNO","DSLINKNO","DSContArea"].
    4) Drops any columns not in the original feature class (except geometry/ID fields).
    5) Writes out the final deduplicated layer to 
       output_reach_gpkg/all_extracted_reaches_{huc_id}.

    Returns:
        Path to the final merged & deduplicated feature class.
    """
    # 1) Load the two inputs
    gdf1 = gpd.read_file(output_reach_gpkg, layer=join_fl_traced_extracted)
    gdf2 = gpd.read_file(output_reach_point_perp_gpkg, layer=join_fl_perp_intersected_filtered)

    # 2) Align CRS if needed
    if gdf1.crs != gdf2.crs:
        gdf2 = gdf2.to_crs(gdf1.crs)

    # 3) Merge row-wise
    merged_gdf = gpd.GeoDataFrame(
        pd.concat([gdf1, gdf2], ignore_index=True),
        crs=gdf1.crs
    )

    # 4) Drop duplicate flowlines
    deduped_gdf = merged_gdf.drop_duplicates(subset=["geometry"])

    # 5) Read original schema to decide which extra fields to drop
    orig_schema = gpd.read_file(original_gpkg, layer=original_layer)
    original_fields = set(orig_schema.columns)

    # Identify fields in deduped_gdf not present in original schema
    fields_to_remove = [fld for fld in deduped_gdf.columns if fld not in original_fields]
    if fields_to_remove:
        deduped_gdf = deduped_gdf.drop(columns=fields_to_remove, errors="ignore")

    # 6) Write out to the specified GPKG
    join_fl_merged = f"join_fl_merged_{huc_id}"
    deduped_gdf.to_file(
        filename=output_reach_gpkg,
        layer=join_fl_merged,
        driver="GPKG"
    )

    print(f"Step 4.3: Merged and deduplicated flowlines saved to {output_reach_gpkg}/{join_fl_merged}")
    return deduped_gdf

In [ ]:
## Step 4.4: Reconstruct disconnected segments again

import geopandas as gpd
import pandas as pd
import numpy as np
import os
import fiona

def reconstruct_disconnected_segments_again(
    unique_id: str,
    unique_id_nextDown: str,
    extracted_layer: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    huc_id: str,
    output_reach_gpkg: str,
    output_final_reach_gpkg: str,
) -> str:
    """
    Pure-Python replacement for reconstruct_disconnected_segments.

    1) Reads the initially extracted network (extract_gdf) and the full join flowline network (join_fl).
    2) Finds NextDownIDs (unique_id_nextDown) that do not appear as HydroIDs in the extracted set.
    3) Traces upstream from each “disconnected end,” adding any missing reaches until hitting a node that is already in the extracted set or a dead-end.
    4) Exports those “missing_reaches” to output_reach_gpkg/missing_reaches_{huc_id}.
    5) Merges them back into the extracted network, writing the final reconstructed network to output_final_reach_gpkg/output_final_reach_gpkg_{huc_id}.
    
    Returns:
        Path to the final reconstructed feature class GeoPackage.
    """
    # Step 0: Read inputs
    extract_gdf = gpd.read_file(output_reach_gpkg, layer=extracted_layer)
    orig_gdf    = gpd.read_file(join_fl_gpkg, layer=join_fl_layer)

    hydro_ids     = extract_gdf[unique_id].to_numpy()
    nextdown_ids  = extract_gdf[unique_id_nextDown].to_numpy()

    # Filter out null NextDownIDs
    valid_mask     = pd.notnull(nextdown_ids)
    nextdown_valid = nextdown_ids[valid_mask]

    disconnected_ends = np.unique(nextdown_valid[~np.isin(nextdown_valid, hydro_ids)])
    disconnected_segments = []

    # Step 1: Trace downstream from each disconnected end
    for end_id in disconnected_ends:
        current_id     = end_id
        missing_reaches = []

        while pd.notnull(current_id):
            match = orig_gdf[orig_gdf[unique_id] == current_id]
            if match.empty:
                break

            if current_id not in hydro_ids:
                missing_reaches.append(current_id)

            # Advance downstream
            current_id = match.iloc[0][unique_id_nextDown]
            if pd.isnull(current_id) or (current_id in hydro_ids):
                break

        if missing_reaches:
            disconnected_segments.append(missing_reaches)

    # Step 2: Export missing reaches
    all_missing_ids = [fid for seg in disconnected_segments for fid in seg]
    if all_missing_ids:
        missing_mask        = orig_gdf[unique_id].isin(all_missing_ids)
        missing_reaches_gdf = orig_gdf[missing_mask].copy()
    else:
        missing_reaches_gdf = orig_gdf.iloc[0:0].copy()  # empty

    merged_missing_reaches_fc = f"join_fl_merged_traced_reaches_{huc_id}"
    missing_reaches_gdf.to_file(output_reach_gpkg, 
                                layer=merged_missing_reaches_fc,
                                driver="GPKG",
                                mode="w")

    # Step 3: Merge and write final
    merged_gdf = gpd.GeoDataFrame(
        pd.concat([extract_gdf, missing_reaches_gdf], ignore_index=True),
        crs=extract_gdf.crs
    )
    merged_gdf = merged_gdf.drop_duplicates(subset=["geometry"])

    join_fl_final = f"join_fl_final_reaches_{huc_id}"

    # # 1) If that layer already exists in the GeoPackage, delete it
    # existing = fiona.listlayers(output_final_reach_gpkg)
    # if join_fl_final in existing:
    #     fiona.remove(output_final_reach_gpkg, layer=join_fl_final)
    #     # print(f"Removed existing layer '{join_fl_final}' from {output_final_reach_gpkg}")
    
    merged_gdf.to_file(output_final_reach_gpkg,
                    layer=join_fl_final,
                    driver="GPKG",
                    mode="w")

    print(f"Step 4.4: Saved reconstructed reaches to {output_final_reach_gpkg}/{join_fl_final}")
    return merged_gdf

In [ ]:
## Step 5: Interactive display of the output flowlines

import geopandas as gpd
import folium

def join_output_interactive_display(
    situation: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    target_fl_gpkg: str,
    target_fl_layer: str,
    output_clip_layer: str,
    output_reach_gpkg: str,
    output_final_reach_gpkg: str,
    output_reach_point_perp_gpkg: str,
    target_reach_unique_id_field: str,
    huc_id: str,
    unique_id: str,
    simplify_tolerance: float = 1000
):
    """
    Interactive display of flowlines by situation, with custom styling:
      - target & clipped:   weight=6,   color=#73B2FF
      - simplified join:    weight=0.5, color=#732600
      - perp_notcross:       weight=1,   color=#000000
      - traced:             weight=3, color=#005CE6
      - perp_filtered:      weight=3, color=yellow
      - final:              weight=3, color=#E64C00

    Adds a layer control to toggle each layer on or off.
    """
    # derive layer names
    perp_notcross  = f"perp_lines_notcross_{huc_id}"
    final_layer    = f"join_fl_final_reaches_{huc_id}"
    perp_filtered  = f"orig_join_perp_notcross_intersected_filtered_{huc_id}"
    traced_layer   = f"join_fl_traced_extracted_reaches_{huc_id}"

    # read & simplify join flowlines
    join_fl   = gpd.read_file(join_fl_gpkg, layer=join_fl_layer)
    join_simp = join_fl.copy()
    join_simp.geometry = join_simp.geometry.simplify(
        tolerance=simplify_tolerance,
        preserve_topology=True
    )

    # base styling for target & clipped
    base_kwargs = dict(
        color="#73B2FF",
        style_kwds={"weight": 6},
        tooltip=target_reach_unique_id_field
    )

    # 1) no_intersection: target then join
    if situation == "no_intersection":
        m = gpd.read_file(target_fl_gpkg, layer=target_fl_layer).explore(
            tiles="cartodbpositron",
            name="Target flowlines",
            **base_kwargs
        )
        join_simp.explore(
            m=m,
            name="Join (simplified)",
            color="#732600",
            style_kwds={"weight": 0.5},
            tooltip=unique_id
        )

    # 2) all_extracted: clipped, join, final
    elif situation == "all_extracted":
        m = gpd.read_file(output_reach_gpkg, layer=output_clip_layer).explore(
            tiles="cartodbpositron",
            name="Clipped target",
            **base_kwargs
        )
        join_simp.explore(
            m=m,
            name="Join (simplified)",
            color="#732600",
            style_kwds={"weight": 0.5},
            tooltip=unique_id
        )
        gpd.read_file(output_final_reach_gpkg, layer=final_layer).explore(
            m=m,
            name="Final reaches",
            color="#E64C00",
            style_kwds={"weight": 3},
            tooltip=unique_id
        )

    # 3) extract_upstream_others: clipped, join, perp_notcross, final, perp_filtered, traced
    elif situation == "extract_upstream_others":
        m = gpd.read_file(output_reach_gpkg, layer=output_clip_layer).explore(
            tiles="cartodbpositron",
            name="Clipped target",
            **base_kwargs
        )
        join_simp.explore(
            m=m,
            name="Join (simplified)",
            color="#732600",
            style_kwds={"weight": 0.5},
            tooltip=unique_id
        )
        gpd.read_file(output_reach_point_perp_gpkg, layer=perp_notcross).explore(
            m=m,
            name="Perp notcross",
            color="#000000",
            style_kwds={"weight": 1},
            tooltip=False
        )
        gpd.read_file(output_final_reach_gpkg, layer=final_layer).explore(
            m=m,
            name="Final reaches",
            color="#E64C00",
            style_kwds={"weight": 3},
            tooltip=unique_id
        )
        gpd.read_file(output_reach_point_perp_gpkg, layer=perp_filtered).explore(
            m=m,
            name="Perp filtered",
            color="yellow",
            style_kwds={"weight": 3},
            tooltip=unique_id
        )
        gpd.read_file(output_reach_gpkg, layer=traced_layer).explore(
            m=m,
            name="Traced reaches",
            color="#005CE6",
            style_kwds={"weight": 3},
            tooltip=unique_id
        )

    else:
        raise ValueError(f"Unknown situation: {situation!r}")

    # add layer control
    folium.LayerControl(collapsed=False).add_to(m)

    # save the map
    m.save(f"output_interactive_map_{situation}_{huc_id}.html")
    print(f"Step 5: Interactive map saved to {os.getcwd()}/output_interactive_map_{situation}_{huc_id}.html")

    return m

In [ ]:
## Step 6: Integrate all the functions and run

from typing import Optional
import os
import geopandas as gpd

def run_workflow(
    huc_id: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    target_fl_gpkg: str,
    target_fl_layer: str,
    unique_id: str,
    unique_id_nextDown: str,
    target_reach_unique_id_field: str,
    *,
    # ← these two are now optional—if None, clip uses join envelope
    vpu_boundary_gpkg: Optional[str] = None,
    vpu_boundary_layer: Optional[str] = None,
    buffer_distance: int = 100, # meters
    spacing: float = 200, # meters
    perp_line_length: float = 600, # meters
    min_join_count: int = 5,
    strm_order_field: Optional[str] = None,
    min_strm_order: int = 2,
    simplify_tolerance: float = 1000
):
    """
    Full RiverJoin workflow. Mandatory arguments:
      - join_fl_gpkg, join_fl_layer
      - target_fl_gpkg, target_fl_layer
      - join_fl: unique_id, unique_id_nextDown, strm_order_field
      - target_reach_unique_id_field
      - huc_id

    Optional settings (passed by keyword only):
      - buffer_distance (e.g. \"100 m\")
      - spacing (float meters or None for fixed count)
      - perp_line_length (float meters)
      - min_join_count (int)
      - min_strm_order (int)
      - simplify_tolerance (float)
    """
    # Step 0.1: Clip SWORD by join flowlines envelope
    output_clip_layer = f"SWORD_VPU_{huc_id}_ByFunction"
    clip_flowlines_by_boundary(
        join_fl_gpkg=join_fl_gpkg,
        join_fl_layer=join_fl_layer,
        target_fl_gpkg=target_fl_gpkg,
        target_fl_layer=target_fl_layer,
        output_clip_gpkg=output_reach_gpkg,
        output_clip_layer=output_clip_layer,
        vpu_boundary_gpkg=vpu_boundary_gpkg,
        vpu_boundary_layer=vpu_boundary_layer
    )

    # Step 1: Extract within buffer
    extract_stream_network_within_sword_buffer(
        huc_id                   = huc_id,
        join_fl_gpkg             = join_fl_gpkg,
        join_fl_layer            = join_fl_layer,
        target_fl_gpkg           = output_reach_gpkg,
        target_fl_layer          = output_clip_layer,
        unique_id                = unique_id,
        output_point_gpkg        = output_point_gpkg,
        output_reach_buffer_gpkg = output_reach_buffer_gpkg,
        output_reach_gpkg        = output_reach_gpkg,
        buffer_distance          = buffer_distance
    )
    extracted_layer = f"join_fl_extracted_reaches_{huc_id}"

    # Step 2: Reconstruct disconnected segments
    reconstruct_disconnected_segments(
        unique_id,
        unique_id_nextDown,
        extracted_layer,
        join_fl_gpkg,
        join_fl_layer,
        huc_id,
        output_reach_gpkg
    )
    join_fl_traced_extracted = f"join_fl_traced_extracted_reaches_{huc_id}"

    # Step 3: Check for further extraction
    situation = process_flowlines(
        extract_stream_gpkg      = output_reach_gpkg,
        extract_layer            = extracted_layer,
        traced_stream_gpkg       = output_reach_gpkg,
        traced_layer             = join_fl_traced_extracted,
        original_gpkg            = join_fl_gpkg,
        original_layer           = join_fl_layer,
        output_final_reach_gpkg  = output_final_reach_gpkg,
        huc_id                   = huc_id
    )

    # Step 4: Upstream‐other branch
    if situation == "extract_upstream_others":
        # 4.0.1: create spaced nodes
        extract_spaced_nodes_projected(
            target_reach_gpkg             = output_reach_gpkg,
            target_reach_layer            = output_clip_layer,
            target_reach_unique_id_field  = target_reach_unique_id_field,
            output_reach_point_perp_gpkg  = output_reach_point_perp_gpkg,
            output_layer                  = f"sword_nodes_space{float_to_tag(spacing)}m_vpu{huc_id}",
            spacing                       = spacing
        )
        # 4.0.2: generate perpendiculars
        extract_perpendicular_lines(
            nodes_gpkg                   = output_reach_point_perp_gpkg,
            nodes_layer                  = f"sword_nodes_space{float_to_tag(spacing)}m_vpu{huc_id}",
            output_reach_point_perp_gpkg = output_reach_point_perp_gpkg,
            perp_line_length             = perp_line_length,
            spacing                      = spacing
        )
        perp_layer = f"perp_lines_len{float_to_tag(perp_line_length)}_space{float_to_tag(spacing)}m_{huc_id}"
        
        # 4.1: remove touching perpendiculars
        remove_touching_perpendiculars(
            output_reach_gpkg               = output_reach_gpkg,
            traced_layer                    = join_fl_traced_extracted,
            output_reach_point_perp_gpkg    = output_reach_point_perp_gpkg,
            perp_layer                      = perp_layer,
            huc_id                          = huc_id
        )
        perp_notcross_layer = f"perp_lines_notcross_{huc_id}"

        # 4.2: spatial join & filter
        perform_spatial_join_with_perp_and_filter(
            huc_id                           = huc_id,
            original_gpkg                    = join_fl_gpkg,
            original_layer                   = join_fl_layer,
            perp_gpkg                        = output_reach_point_perp_gpkg,
            perp_notcross_layer              = perp_notcross_layer,
            unique_id                        = unique_id,
            output_reach_point_perp_gpkg     = output_reach_point_perp_gpkg,
            min_join_count                   = min_join_count,
            strm_order_field                 = strm_order_field,
            min_strm_order                   = min_strm_order
        )
        join_fl_perp_intersected_filtered = f"orig_join_perp_notcross_intersected_filtered_{huc_id}"

        # 4.3: merge & dedupe
        merge_and_drop_duplicates(
            join_fl_perp_intersected_filtered = join_fl_perp_intersected_filtered,
            join_fl_traced_extracted          = join_fl_traced_extracted,
            original_gpkg                     = join_fl_gpkg,
            original_layer                    = join_fl_layer,
            output_reach_point_perp_gpkg      = output_reach_point_perp_gpkg,
            output_reach_gpkg                 = output_reach_gpkg,
            huc_id                            = huc_id
        )
        join_fl_merged = f"join_fl_merged_{huc_id}"

        # 4.4: reconnect segments again
        reconstruct_disconnected_segments_again(
            unique_id,
            unique_id_nextDown,
            join_fl_merged,
            join_fl_gpkg,
            join_fl_layer,
            huc_id,
            output_reach_gpkg,
            output_final_reach_gpkg
        )
    else:
        print(f"No intersection for VPU {huc_id}. Skipping Step 4.")

    # Step 5: Interactive display
    m = join_output_interactive_display(
        situation,
        join_fl_gpkg,
        join_fl_layer,
        target_fl_gpkg,
        target_fl_layer,
        output_clip_layer,
        output_reach_gpkg,
        output_final_reach_gpkg,
        output_reach_point_perp_gpkg,
        target_reach_unique_id_field,
        huc_id,
        unique_id,
        simplify_tolerance             = simplify_tolerance
    )

    print("Workflow complete.")

    # return it so that a naked call also renders the map
    return m

## Run the next cell to do the spatial join of SWORD stream networks and the GEOGLOWS stream networks
### [Specify your own layer paths and fields; for the other parameter, specify or leave them default]

In [ ]:
# Example call:
huc_id = "715"

run_workflow(
    huc_id                      = huc_id,
    join_fl_gpkg                = f"/Volumes/MacOS/UA/Dataset/GEOGLOWS/streams_{huc_id}_v2.gpkg",
    join_fl_layer               = f"streams_{huc_id}",
    target_fl_gpkg              = "/Volumes/MacOS/UA/Dataset/SWORD/SWORD_v17b_gpkg/na_sword_reaches_v17b.gpkg",
    target_fl_layer             = "reaches",
    unique_id                   = "LINKNO",
    unique_id_nextDown          = "DSLINKNO",
    target_reach_unique_id_field= "reach_id",

    ## Optional VPU boundary
    ## If None, will use join flowlines envelope for clipping
    # vpu_boundary_gpkg           = "/Volumes/MacOS/UA/Dataset/RiverJoin/GDBs_GEOGLOWS_SWORD/OtherData.gpkg",
    # vpu_boundary_layer          = f"GEOGLOWS_VPU_{huc_id}",

    ## Optional parameters
    # buffer_distance             = 100,         # meters, buffer distance of the target flowlines in Step 1
    # spacing                     = 200,         # meters, spacing for nodes in Step 4.0.1
    # perp_line_length            = 600,         # meters, length of perpendicular lines in Step 4.0.2
    # min_join_count              = 5,           # minimum number of intersections to keep in Step 4.2
    strm_order_field            = "strmOrder",   # stream order field to filter by in Step 4.2
    # min_strm_order              = 2,           # minimum stream order to keep in Step 4.2
    # simplify_tolerance          = 1000         # meters, tolerance for simplifying geometries in Step 5
)

In [ ]:
## Step 7.0: Optional Function, if user calls compare_line_lengths, then execute it
## Make it as a separate function, if the user call, then excute it, otherwise, just define it

import geopandas as gpd
import matplotlib.pyplot as plt

def pick_utm_epsg(gdf: gpd.GeoDataFrame) -> int:
    """Pick the appropriate UTM zone EPSG code from a layer’s centroid."""
    minx, miny, maxx, maxy = gdf.total_bounds
    midx, midy = (minx + maxx) / 2, (miny + maxy) / 2
    zone = int((midx + 180) // 6) + 1
    return 32600 + zone if midy >= 0 else 32700 + zone

def print_basic_stats(name: str, series) -> None:
    """
    Print count, mean, std, min, 25%, 50%, 75%, max for the given numeric Series.
    """
    stats = series.describe().round(2)
    print(f"\n{name} (metres):")
    print(stats)

def compare_line_lengths(
    join_gpkg: str,
    join_layer: str,
    target_gpkg: str,
    target_layer: str
) -> None:
    """
    1) Read two line layers from their GeoPackages.
    2) Harmonize their CRS:
       • If one is projected & the other geographic → reproject the geographic one.
       • If both geographic → pick UTM from target & reproject both.
       • If both projected → reproject join into target’s CRS.
    3) Compute each line’s length in metres.
    4) Print basic summary statistics for each set of lengths.
    """
    # --- 1) Read layers ---
    join_fl   = gpd.read_file(join_gpkg,   layer=join_layer)
    target_fl = gpd.read_file(target_gpkg, layer=target_layer)

    # --- 2) Harmonize CRS ---
    crs1, crs2   = join_fl.crs,    target_fl.crs
    geo1, geo2   = crs1.is_geographic, crs2.is_geographic

    if not geo1 and geo2:
        join_proj, target_proj = join_fl, target_fl.to_crs(crs1)
    elif geo1 and not geo2:
        join_proj, target_proj = join_fl.to_crs(crs2), target_fl
    elif geo1 and geo2:
        epsg_utm     = pick_utm_epsg(target_fl)
        join_proj    = join_fl.to_crs(epsg=epsg_utm)
        target_proj  = target_fl.to_crs(epsg=epsg_utm)
    else:
        join_proj, target_proj = join_fl.to_crs(crs2), target_fl

    # --- 3) Compute lengths (metres in projected CRS) ---
    join_lengths   = join_proj.geometry.length
    target_lengths = target_proj.geometry.length

    # --- 4) Print stats ---
    print_basic_stats("Join line lengths",   join_lengths)
    print_basic_stats("Target line lengths", target_lengths)

## Optionally run the next cell to have an overall impression of the lengths of the SWORD networks and the extracted GEOGLOWS ones
### This may assist to decide how many nodes along your each flowline to use to join the corresponding joining flowline(s) (e.g. GEOGLOWS here) attribute

In [ ]:
output_clip_layer = f"SWORD_VPU_{huc_id}_ByFunction"
join_fl_final = f"join_fl_final_reaches_{huc_id}"

compare_line_lengths(
    join_gpkg   = output_final_reach_gpkg,
    join_layer  = join_fl_final,
    target_gpkg = output_reach_gpkg,
    target_layer= output_clip_layer,
)

In [ ]:
## Step 7.1: Attributes translation from the join_fl to the target_fl
## Step 7.1.1: Generate the nodes from the target flowlines

import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from shapely.ops import linemerge

def extract_spaced_or_numbered_nodes_projected(
    target_reach_gpkg: str,
    target_reach_layer: str,
    target_reach_unique_id_field: str,
    output_final_reach_joined_gpkg: str,
    output_layer: str,
    spacing: float = None,
    node_number: int = None
) -> gpd.GeoDataFrame:
    """
    Read a line layer and interpolate nodes either by spacing or by fixed node count.
    If neither provided, defaults to node_number=5. node_number must be ≥3.
    """

    # 0) Default & validate modes/types
    if spacing is None and node_number is None:
        node_number = 5

    if spacing is not None and node_number is not None:
        raise ValueError(
            f"Specify exactly one of `spacing` or `node_number`, not both.\n"
            f"  spacing={spacing!r} (type: {type(spacing).__name__}), "
            f"node_number={node_number!r} (type: {type(node_number).__name__})."
        )

    if spacing is not None and not isinstance(spacing, (int, float)):
        raise TypeError(
            f"`spacing` must be a number (int or float), got {spacing!r} "
            f"(type: {type(spacing).__name__})."
        )

    if node_number is not None:
        if not isinstance(node_number, int):
            raise TypeError(
                f"`node_number` must be an integer, got {node_number!r} "
                f"(type: {type(node_number).__name__})."
            )
        if node_number < 3:
            raise ValueError(
                f"`node_number` must be at least 3, got {node_number!r} "
                f"(type: {type(node_number).__name__})."
            )

    # ... rest of function unchanged ...
    lines = (
        gpd.read_file(target_reach_gpkg, layer=target_reach_layer)
           .dropna(subset=["geometry"])
        #    .explode(ignore_index=True)
    )
    orig_crs = lines.crs

    # --- CHANGED: merge multipart geometries into single LineString per feature ---
    lines["geometry"] = lines.geometry.apply(
        lambda g: linemerge(g) if isinstance(g, MultiLineString) else g
    )

    # Reproject to UTM if necessary
    if orig_crs.is_geographic:
        minx, miny, maxx, maxy = lines.total_bounds
        midx, midy = (minx + maxx)/2, (miny + maxy)/2
        zone = int((midx + 180)//6) + 1
        epsg_utm = 32600 + zone if midy >= 0 else 32700 + zone
        lines_proj = lines.to_crs(epsg=epsg_utm)
        need_reproject_back = True
    else:
        lines_proj = lines
        need_reproject_back = False

    records = []
    for row in lines_proj.itertuples():
        seg = row.geometry  # always a single LineString now
        L = seg.length
        if L == 0:
            continue

        if spacing is not None:
            q = L / spacing
            n = int(np.floor(q)) + 1 if q >= 3 else 3
        else:
            n = node_number

        fracs = np.linspace(0, 1, n)
        for f in fracs:
            pt = seg.interpolate(f * L)
            records.append({
                f"{target_reach_unique_id_field}": getattr(row, target_reach_unique_id_field, None),
                "geometry": pt
            })

    pts_proj = gpd.GeoDataFrame(records, crs=lines_proj.crs)
    pts = pts_proj.to_crs(orig_crs) if need_reproject_back else pts_proj

    # os.makedirs(os.path.dirname(output_final_reach_joined_gpkg), exist_ok=True)
    pts.to_file(
        filename=output_final_reach_joined_gpkg,
        layer=output_layer,
        driver="GPKG"
    )

    mode = "spacing→nodes" if spacing is not None else "fixed nodes"
    print(f"→ Generated and wrote {len(pts)} nodes ({mode}) "
          f"to {output_final_reach_joined_gpkg}/{output_layer}")
    return pts

In [ ]:
## Step 7.1.2: Join nearest flowline attributes to the nodes and then back to the target flowlines

import os
import geopandas as gpd
from typing import List

import os
import geopandas as gpd

def join_nearest_join_fl_attributes(
    nodes_gpkg: str,
    nodes_layer: str,
    join_fl_gpkg: str,
    join_fl_layer: str,
    target_fl_gpkg: str,
    target_fl_layer: str,
    common_id: str,
    output_gpkg: str,
    output_nodes_layer: str,
    output_flowlines_layer: str
) -> gpd.GeoDataFrame:
    """
    1) Read node points and flowlines.
    2) Reproject join_fl into nodes CRS if they differ.
    2b) Compute a 'reach_length_for_join' (in metres) on join_fl:
         - if geographic, reproject to UTM, measure, then keep in original CRS
         - else measure directly
    3) Nearest‐geometry join: attach all flowline attributes to each node.
    4) Drop the flowline geometry from the node table.
    5) Join those node attributes back onto the target flowlines by `common_id`.
    6) Write out:
       - nodes_with_attrs layer to output_gpkg/output_nodes_layer
       - enriched flowlines layer to output_gpkg/output_flowlines_layer
       - enriched flowlines (no geometry) as CSV alongside the GPKG
    """
    # 1) Load inputs
    nodes     = gpd.read_file(nodes_gpkg,    layer=nodes_layer)
    join_fl   = gpd.read_file(join_fl_gpkg,  layer=join_fl_layer)
    target_fl = gpd.read_file(target_fl_gpkg, layer=target_fl_layer)

    # 2) Align CRS if necessary
    if join_fl.crs != nodes.crs:
        join_fl = join_fl.to_crs(nodes.crs)
    
    # 2b) Compute reach_length_for_join (in metres)
    aligned_crs = join_fl.crs
    if aligned_crs.is_geographic:
        # pick UTM zone based on join_fl extent
        minx, miny, maxx, maxy = join_fl.total_bounds
        midx, midy = (minx + maxx) / 2, (miny + maxy) / 2
        zone = int((midx + 180) // 6) + 1
        utm_epsg = 32600 + zone if midy >= 0 else 32700 + zone
        tmp = join_fl.to_crs(epsg=utm_epsg)
        join_fl['reach_length_for_join'] = tmp.geometry.length
    else:
        join_fl['reach_length_for_join'] = join_fl.geometry.length

    # 3) Nearest‐geometry spatial join
    joined_nodes = gpd.sjoin_nearest(
        nodes,
        join_fl,
        how="left",
        distance_col="dist_to_line"
    )
    joined_nodes = joined_nodes.drop(columns="geometry_right", errors="ignore")

    # 4) Join node attributes back to the flowlines
    node_attrs = joined_nodes.drop(columns=["geometry"], errors="ignore")
    flowlines_with_attrs = target_fl.merge(
        node_attrs,
        on=common_id,
        how="left"
    )

    # 6) Write out
    os.makedirs(os.path.dirname(output_gpkg), exist_ok=True)
    joined_nodes.to_file(
        filename=output_gpkg,
        layer=output_nodes_layer,
        driver="GPKG"
    )
    flowlines_with_attrs.to_file(
        filename=output_gpkg,
        layer=output_flowlines_layer,
        driver="GPKG"
    )
    # write CSV of flowlines attributes only
    csv_dir = os.path.join(os.path.dirname(output_gpkg), "FinalReaches_JoinedCommonID_csv")
    os.makedirs(csv_dir, exist_ok=True)
    csv_path = os.path.join(csv_dir, f"{output_flowlines_layer}.csv")
    flowlines_with_attrs.drop(columns="geometry", errors="ignore") \
        .to_csv(csv_path, index=False)

    print(f"→ Wrote {len(joined_nodes)} nodes to {output_gpkg}/{output_nodes_layer}")
    print(f"→ Wrote {len(flowlines_with_attrs)} flowlines to {output_gpkg}/{output_flowlines_layer}")
    print(f"→ Wrote CSV of flowlines attributes to {csv_path}")

    return flowlines_with_attrs

In [ ]:
## Step 7.2: Aggregate the joined flowlines to one per target flowline, based on the count of joined flowlines

import os
import geopandas as gpd

def aggregate_joined_flowlines(
    joined_output_gpkg: str,
    joined_output_layer: str,
    target_fl_unique_id_field: str,
    join_fl_unique_id_field: str,
    output_aggregated_layer: str
) -> gpd.GeoDataFrame:
    """
    1) Read `joined_output_layer` from `joined_output_gpkg`, containing multiple joined records per `target_fl_unique_id_field`.
    2) Count how often each `join_fl_unique_id_field` occurs for each `target_fl_unique_id_field`.
    3) Keep only the flowline with the highest count; tie-break by the 'reach_length_for_join' field.
    4) Merge back full-feature records, drop duplicates.
    5) Drop the 'reach_length_for_join' field.
    6) Write aggregated layer back to GeoPackage under `output_aggregated_layer`.
    7) Export attributes (no geometry) as CSV.
    """
    # 1) Read the joined features
    gdf = gpd.read_file(joined_output_gpkg, layer=joined_output_layer)

    # 2) Count occurrences
    counts = (
        gdf
        .groupby([target_fl_unique_id_field, join_fl_unique_id_field])
        .size()
        .reset_index(name='cnt')
    )
    counts['max_cnt'] = counts.groupby(target_fl_unique_id_field)['cnt'].transform('max')
    tied = counts[counts['cnt'] == counts['max_cnt']].copy()

    # 3) Tie-breaking by reach_length_for_join
    lengths = (
        gdf[[join_fl_unique_id_field, 'reach_length_for_join']]
           .drop_duplicates(subset=join_fl_unique_id_field)
    )
    tied = tied.merge(lengths, on=join_fl_unique_id_field, how='left')
    idx = tied.groupby(target_fl_unique_id_field)['reach_length_for_join'].idxmax()
    chosen = tied.loc[idx, [target_fl_unique_id_field, join_fl_unique_id_field]]

    # 4) Merge back and dedupe
    aggregated = (
        gdf
        .merge(chosen, on=[target_fl_unique_id_field, join_fl_unique_id_field], how='inner')
        .drop_duplicates(subset=[target_fl_unique_id_field, join_fl_unique_id_field])
    )

    # 5) Drop the helper length column
    aggregated = aggregated.drop(columns=['reach_length_for_join'], errors='ignore')

    # 6) Write aggregated layer
    aggregated.to_file(
        filename=joined_output_gpkg,
        layer=output_aggregated_layer,
        driver='GPKG',
        mode='w'
    )
    print(f"Aggregated {len(aggregated)} unique records "
          f"to layer '{output_aggregated_layer}' in {joined_output_gpkg}")

    # 7) Export attributes to CSV
    joined_output_csv_folder = os.path.join(os.path.dirname(joined_output_gpkg), "FinalReaches_JoinedCommonID_csv")
    csv_path = os.path.join(joined_output_csv_folder, f"{output_aggregated_layer}.csv")
    aggregated.drop(columns='geometry').to_csv(csv_path, index=False)
    print(f"Attribute table saved as CSV to {csv_path}")

    return aggregated

In [ ]:
# Step 7.2.1: Average aggregation, can only be used for numeric fields
# TBD...


In [ ]:
## Step 8: Put all the data translation functions together

import os
import geopandas as gpd

def process_final_node_join_aggregation(
    huc_id: str,
    output_final_reach_gpkg: str,
    join_fl_final_layer: str,
    output_reach_gpkg: str,
    output_clip_layer: str,
    output_final_reach_joined_gpkg: str,
    target_fl_unique_id_field: str,
    join_fl_unique_id_field: str,
    spacing: float | None = None,
    node_number: int | None = None
) -> gpd.GeoDataFrame:
    """
    Runs the full three‐step sequence to:
      1) Generate spaced (or numbered) nodes along the final reaches.
      2) Nearest‐join flowline attributes to those nodes and back to reaches.
      3) Aggregate to one record per reach by counting & longest‐reach tie‐break.
    Returns the final aggregated GeoDataFrame.
    """
    # 1) Generate nodes
    nodes_layer = f"join_fl_final_reaches_nodes_vpu{huc_id}"
    extract_spaced_or_numbered_nodes_projected(
        target_reach_gpkg               = output_final_reach_gpkg,
        target_reach_layer              = join_fl_final_layer,
        target_reach_unique_id_field    = target_fl_unique_id_field,
        output_final_reach_joined_gpkg  = output_final_reach_joined_gpkg,
        output_layer                    = nodes_layer,
        spacing                         = spacing,
        node_number                     = node_number
    )

    # 2) Join nearest flowline attributes
    joined_nodes_layer   = f"join_fl_final_reaches_nodes_joinedAttr_vpu{huc_id}"
    joined_flowline_layer    = f"join_fl_final_reaches_joinedAttr_vpu{huc_id}"
    join_nearest_join_fl_attributes(
        nodes_gpkg           = output_final_reach_joined_gpkg,
        nodes_layer          = nodes_layer,
        join_fl_gpkg         = output_reach_gpkg,
        join_fl_layer        = output_clip_layer,
        target_fl_gpkg       = output_final_reach_gpkg,
        target_fl_layer      = join_fl_final_layer,
        common_id            = target_fl_unique_id_field,
        output_gpkg          = output_final_reach_joined_gpkg,
        output_nodes_layer   = joined_nodes_layer,
        output_flowlines_layer = joined_flowline_layer
    )

    # 3) Aggregate to one reach record
    aggregated_layer = f"join_fl_final_reaches_joinedAttr_agg_vpu{huc_id}"
    aggregated = aggregate_joined_flowlines(
        joined_output_gpkg            = output_final_reach_joined_gpkg,
        joined_output_layer           = joined_flowline_layer,
        target_fl_unique_id_field     = target_fl_unique_id_field,
        join_fl_unique_id_field       = join_fl_unique_id_field,
        output_aggregated_layer       = aggregated_layer
    )

    print("→ Completed node generation, join, and aggregation.")
    return aggregated

## Run the next cell to do data transfer, here we join the unique id of corresponding GEOGLOWS flowline to the SWORD flowline

In [ ]:
# huc_id = "715"

# Determine base directory: script folder if running as a file, else cwd
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.getcwd()

final_gdf = process_final_node_join_aggregation(
    huc_id                        = huc_id,
    output_final_reach_gpkg       = f"{script_dir}/RiverJoin/output/TempReaches.gpkg",
    join_fl_final_layer           = f"SWORD_VPU_{huc_id}_ByFunction",
    output_reach_gpkg             = f"{script_dir}/RiverJoin/output/FinalReaches.gpkg",
    output_clip_layer             = f"join_fl_final_reaches_{huc_id}",
    output_final_reach_joined_gpkg= f"{script_dir}/RiverJoin/output/FinalReaches_JoinedCommonID.gpkg",
    target_fl_unique_id_field     = "reach_id",
    join_fl_unique_id_field       = "LINKNO",

    ## Optional parameters
    # spacing                       = 1000,    # meters, or None if using node_number
    # node_number                   = 5,
)